In [6]:
import os,boto3
bucket_name_text= os.environ['LEARNER_S3_BUCKETNAME_TEXT'] = 'bedrockproject'
os.environ['LAMBDALAYERVERSIONARN'] = "arn:aws:lambda:us-west-2:332848115362:layer:dlai-bedrock-jinja-layer:1"
bucket_name_audio = os.environ['LEARNER_S3_BUCKETNAME_AUDIO'] = 'transcribeaudio-bucket'

In [7]:
# We are going to make a fully automated architecture
# Put the audio in an audio bucket transcribe it and put it in the text bucket
import boto3, os

from helpers.Lambda_Helper import Lambda_Helper
from helpers.S3_Helper import S3_Helper

lambda_helper = Lambda_Helper()
s3_helper = S3_Helper()

In [8]:
%%writefile lambda_function.py

import json
import boto3
import uuid
import os

s3_client = boto3.client('s3')
transcribe_client = boto3.client('transcribe',region_name='us-west-2')

def lambda_handler(event,context):
    # Extracting the bucket name and key from incoming event
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['bucket']['name']

    if key != 'dialog.mp3':
        print("This demo only works with dialog.mp3.")
        return
    
    try:

        job_name = 'transcription-job-' + str(uuid.uuid4())

        response = transcribe_client.start_transcription_job(
            TranscriptionJobName = job_name,
            Media = {'MediaFileUri' : f's3://{bucket}/{key}'},
            MediaFormat = 'mp3',
            LanguageCode = 'en-US',
            OutputBucketName = os.environ['S3BUCKETNAMETEXT'],
            OutputKey=f'{job_name}-transcript.json',
            Settings = {
                'ShowSpeakerLabels':True,
                'MaxSpeakerLabels':2
            }

        )
    
    except Exception as e:
        print(f"Error occurred: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Error occurred: {e}")
        }

    return {
        'statusCode': 200,
        'body': json.dumps(f"Submitted transcription job for {key} from bucket {bucket}.")
    }


Overwriting lambda_function.py


In [9]:
lambda_helper.lambda_environ_variables = {'S3BUCKETNAMETEXT' : bucket_name_text}
lambda_helper.deploy_function(["lambda_function.py"], function_name="LambdaFunctionTranscribe")

Zipping function...
Looking for existing function...
Function LambdaFunctionTranscribe does not exist. Creating...
Function LambdaFunctionTranscribe created: arn:aws:lambda:us-west-2:332848115362:function:LambdaFunctionTranscribe
Done.


In [10]:
# Trigger is an mp3 file
s3_helper.upload_file(bucket_name_audio, 'dialog.mp3')

Object 'dialog.mp3' uploaded to bucket 'transcribeaudio-bucket'


In [11]:
s3_helper.list_objects(bucket_name_audio)
# We check the audio bucket

Object: dialog.mp3, Created on: 2024-03-20 11:39:16+00:00


In [12]:
# Now we check the text bucket and what it contains
s3_helper.list_objects(bucket_name_text)

Object: .write_access_check_file.temp, Created on: 2024-03-03 10:02:53+00:00
Object: dialog.mp3, Created on: 2024-03-03 10:02:25+00:00
Object: transcription-job-6230cc89-e538-4168-a681-61201b25d1aa.json, Created on: 2024-03-03 10:03:11+00:00


In [13]:
s3_helper.download_object(bucket_name_text, 'results.txt')

Error: An error occurred (404) when calling the HeadObject operation: Not Found
